In [21]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import torch
import cvxpy as cp
from cvxpylayers.torch import CvxpyLayer
import yfinance as yf
import random
from dataclasses import dataclass
from typing import Dict, Tuple
from tqdm import tqdm
import itertools

### Simulation 1 : Estimation Error in $\hat{V}$

In [2]:
SIM1_CFG = dict(
    sigma=0.0125,
    rho_grid=[0.0, 0.25, 0.5, 0.75],
    d_z=10,
    d_x=10,
    res_grid=[5, 10, 20],
    snr_grid=[0.001, 0.002, 0.003, 0.004, 0.005, 0.01, 0.05, 0.10],
    n_total=2000,
    n_train=1000,
    n_test=1000,
    n_reps=100,
    delta=1.0,
    cov_structure="toeplitz_rho_absdiff",
    x_dist="normal",
    theta0_dist="normal",
    snr_matching="scale_tau_per_snr",
    mvo_case="equality",          # 1^T z = 1
    vhat_mode="single_per_split", # estimation error 주입 방식
    vhat_estimator="sample_cov",
)
SIM1_CFG["s_grid"] = [r * SIM1_CFG["d_z"] for r in SIM1_CFG["res_grid"]]


In [3]:
def make_V(d, sigma, rho):
    idx = torch.arange(d)
    dist = (idx[:, None] - idx[None, :]).abs()
    V = (sigma ** 2) * (rho ** dist)
    return V



def compute_tau(signal, V, target_snr):
    # signal = (n,d) / Var(d, d)
    var_signal = signal.var(unbiased = False)
    var_eps = torch.trace(V) / V.shape[0]
    tau = torch.sqrt(var_signal / (target_snr * var_eps))
    return tau


def generate_sim1(cfg, rho, snr, s_cov, seed = 0, device = 'cpu'):
    torch.manual_seed(seed)

    d = cfg["d_z"]
    n_total = cfg["n_total"]
    n_train = cfg["n_train"]

    # (1) Ground Truth Coefficients
    theta0 = torch.randn(d, device = device)

    # (2) x ~ N(0, I)
    x = torch.randn(n_total, d, device = device)
    V = make_V(d, cfg["sigma"], rho).to(device)

    # (3) tau 생성, eps ~ N(0, V) 샘플링
    signal = x * theta0
    tau = compute_tau(signal, V, snr)

    L = torch.linalg.cholesky(V)
    z = torch.randn(n_total, d, device = device)
    eps = z @ L.T  

    y = signal + tau * eps

    # (4) Train-Test Split
    x_train, y_train = x[:n_train], y[:n_train]
    x_test, y_test = x[n_train:], y[n_train:]

    # (5) Generate Estimates of V
    idx = torch.randperm(n_train, device = device)[:s_cov]
    Y_s = y_train[idx]
    Yc = Y_s - Y_s.mean(dim = 0, keepdim = True)
    Vhat = (Yc.T @ Yc) / (s_cov - 1)

    return V, Vhat, theta0, x_train, y_train, x_test, y_test, tau

##### 실행 예시

In [4]:
cfg = SIM1_CFG

V, Vhat, theta0, x_tr, y_tr, x_te, y_te, tau = generate_sim1(
    cfg = cfg,
    rho = 0.5,
    snr = 0.005,
    s_cov = 100,
    seed = 42
)

print("V_shape: ", V.shape)
print("Vhat shape: ", Vhat.shape)
print("tau: ", tau.item())
print("train: ", x_tr.shape, y_tr.shape, "test: ", x_te.shape, y_te.shape)

signal_tr = x_tr * theta0
noise_tr  = y_tr - signal_tr

emp_snr = signal_tr.var(unbiased=False) / noise_tr.var(unbiased=False)
print("Empirical SNR:", emp_snr.item())

V_shape:  torch.Size([10, 10])
Vhat shape:  torch.Size([10, 10])
tau:  956.8330078125
train:  torch.Size([1000, 10]) torch.Size([1000, 10]) test:  torch.Size([1000, 10]) torch.Size([1000, 10])
Empirical SNR: 0.004950998816639185


### 논문 수식

\[
z^*(\hat y)
=
\frac{1}{\delta} \, G \hat y
+
\left(I - G \hat V\right) z_0
\]

\[
G
=
F \left(F^\top \hat V F\right)^{-1} F^\top
\]


In [9]:
def fit_ols_univariate(x_train, y_train, eps = 1e-12):
    num = (x_train * y_train).sum(dim = 0)
    den = (x_train * x_train).sum(dim = 0)
    theta_hat = num / (den + eps)

    return theta_hat



def predict_univariate(x, theta_hat):
    return x * theta_hat



def make_F_and_z0(d, device= 'cpu'):
    one = torch.ones(d, device = device)
    z0 = one / d

    R = torch.randn(d, d-1, device = device)  #d-1 차원은 등호 제약식 하나 때문에
    R = R - one[:, None] * (one @ R) / (one @ one)
    Q, _ = torch.linalg.qr(R)
    F = Q

    return F, z0


def fit_ipo_eq(x_train, y_train, V, Vhat, delta = 1.0, ridge = 1e-10):
    device = x_train.device
    m, d = x_train.shape

    F, z0 = make_F_and_z0(d, device = device)
    mid = torch.linalg.inv(F.T @ Vhat @ F)
    G = F @ mid @ F.T

    I = torch.eye(d, device = device)

    A = G @ V @ G
    b = V @ (I - G @ Vhat) @ z0

    Heq = torch.zeros(d, d, device = device)
    deq = torch.zeros(d, device = device)

    for i in range(m):
        x = x_train[i]
        y = y_train[i]

        # i = 1 ~ m에 대해 Heq, deq 누적 계산 (표본 평균 때문)
        Heq += (x[:, None] * A * x[None, :])
        deq += x * (G @ (y - b))

    Heq = Heq / (m * delta)
    deq = deq / (m * delta)

    Heq = Heq + ridge * torch.eye(d, device=device)

    theta_ipo = torch.linalg.solve(Heq, deq)
    return theta_ipo

In [17]:
def compute_G_and_z0(d, Vhat, device = 'cpu'):
    F, z0 = make_F_and_z0(d, device=device)
    mid = torch.linalg.inv(F.T @ Vhat @ F)
    G = F @ mid @ F.T

    return G, z0


def mvo_eq(yhat, Vhat, delta = 1.0):

    device = yhat.device
    n, d = yhat.shape

    G, z0 = compute_G_and_z0(d, Vhat, device=device)
    I = torch.eye(d, device = device)

    # z* = (1/delta) G yhat + (I - G Vhat) z0
    term1 = (yhat @ G.T) / delta
    term2 = ((I - G @ Vhat) @ z0).view(1, -1)
    zstar = term1 + term2

    return zstar


def decision_cost(z, y, V, delta = 1.0):
    term_ret = -(z * y).sum(dim = 1)

    Vz = z @ V
    term_risk = 0.5 * delta * (Vz * z).sum(dim = 1)

    return term_ret + term_risk


def pve_loss(y, yhat):
    sse = ((y - yhat) ** 2).mean()
    sst = ((y - y.mean(dim=0, keepdim=True)) ** 2).mean()
    pve = (1 - sse/sst).item()

    return pve

In [22]:
def run_sim1_grid(cfg, device="cpu", base_seed=0):
    results = []

    # 모든 (rho, s_cov, snr) 조합 생성
    grid = list(itertools.product(
        cfg["rho_grid"],
        cfg["s_grid"],
        cfg["snr_grid"]
    ))

    total_cases = len(grid)  # 96
    print(f"Total grid cases: {total_cases}")

    # tqdm으로 감싸기
    for rho, s_cov, snr in tqdm(grid, total=total_cases, desc="Grid Progress"):

        cost_ols_list = []
        cost_ipo_list = []
        pve_ols_list  = []
        pve_ipo_list  = []

        for rep in range(cfg["n_reps"]):
            seed = base_seed + rep

            # (i)~(v)
            V, Vhat, theta0, x_tr, y_tr, x_te, y_te, tau = generate_sim1(
                cfg=cfg, rho=rho, snr=snr, s_cov=s_cov, seed=seed, device=device
            )

            # (vi)
            theta_ols = fit_ols_univariate(x_tr, y_tr)
            theta_ipo = fit_ipo_eq(x_tr, y_tr, V, Vhat, delta=cfg["delta"])

            # (vii)
            yhat_ols = predict_univariate(x_te, theta_ols)
            yhat_ipo = predict_univariate(x_te, theta_ipo)

            z_ols = mvo_eq(yhat_ols, Vhat, delta=cfg["delta"])
            z_ipo = mvo_eq(yhat_ipo, Vhat, delta=cfg["delta"])

            # (viii)
            c_ols = decision_cost(z_ols, y_te, V, delta=cfg["delta"]).mean().item()
            c_ipo = decision_cost(z_ipo, y_te, V, delta=cfg["delta"]).mean().item()

            cost_ols_list.append(c_ols)
            cost_ipo_list.append(c_ipo)
            pve_ols_list.append(pve_loss(y_te, yhat_ols))
            pve_ipo_list.append(pve_loss(y_te, yhat_ipo))

        # 평균
        cost_ols_mean = sum(cost_ols_list) / len(cost_ols_list)
        cost_ipo_mean = sum(cost_ipo_list) / len(cost_ipo_list)
        pve_ols_mean  = sum(pve_ols_list)  / len(pve_ols_list)
        pve_ipo_mean  = sum(pve_ipo_list)  / len(pve_ipo_list)

        results.append({
            "rho": rho,
            "s_cov": s_cov,
            "snr": snr,
            "cost_ols": cost_ols_mean,
            "cost_ipo": cost_ipo_mean,
            "cost_ratio_ipo_over_ols": cost_ipo_mean / cost_ols_mean,
            "pve_ols": pve_ols_mean,
            "pve_ipo": pve_ipo_mean,
            "pve_diff_ipo_minus_ols": pve_ipo_mean - pve_ols_mean,
        })

    return pd.DataFrame(results)


# 실행
df = run_sim1_grid(SIM1_CFG, device="cpu", base_seed=42)

Total grid cases: 96


Grid Progress: 100%|██████████| 96/96 [01:25<00:00,  1.13it/s]

   rho  s_cov    snr  cost_ols      cost_ipo  cost_ratio_ipo_over_ols  \
0  0.0     50  0.001  0.014582  -5657.049115            -3.879446e+05   
1  0.0     50  0.002 -0.003877 -11107.081936             2.865165e+06   
2  0.0     50  0.003 -0.018087 -14338.607852             7.927460e+05   
3  0.0     50  0.004 -0.030906 -16346.199989             5.289079e+05   
4  0.0     50  0.005 -0.043048 -17686.185060             4.108518e+05   

    pve_ols       pve_ipo  pve_diff_ipo_minus_ols  
0 -0.001147 -1.858166e+10           -1.858166e+10  
1 -0.000162 -1.341524e+10           -1.341524e+10  
2  0.000823 -9.642090e+09           -9.642090e+09  
3  0.001809 -7.352327e+09           -7.352327e+09  
4  0.002793 -5.888132e+09           -5.888132e+09  


In [23]:
df

,rho,s_cov,snr,cost_ols,cost_ipo,cost_ratio_ipo_over_ols,pve_ols,pve_ipo,pve_diff_ipo_minus_ols
0,0.00,50,0.001,0.014582,-5657.049115,-3.879446e+05,-0.001147,-1.858166e+10,-1.858166e+10
1,0.00,50,0.002,-0.003877,-11107.081936,2.865165e+06,-0.000162,-1.341524e+10,-1.341524e+10
2,0.00,50,0.003,-0.018087,-14338.607852,7.927460e+05,0.000823,-9.642090e+09,-9.642090e+09
3,0.00,50,0.004,-0.030906,-16346.199989,5.289079e+05,0.001809,-7.352327e+09,-7.352327e+09
4,0.00,50,0.005,-0.043048,-17686.185060,4.108518e+05,0.002793,-5.888132e+09,-5.888132e+09
...,...,...,...,...,...,...,...,...,...
91,0.75,200,0.004,-0.103780,-88159.346230,8.494858e+05,0.001703,-1.129210e+10,-1.129210e+10
92,0.75,200,0.005,-0.137334,-90094.373086,6.560247e+05,0.002678,-9.229211e+09,-9.229211e+09
93,0.75,200,0.010,-0.291722,-94211.346113,3.229492e+05,0.007541,-4.985706e+09,-4.985706e+09
94,0.75,200,0.050,-1.184015,-96605.838145,8.159173e+04,0.044999,-1.660607e+09,-1.660607e+09
